# Eat Safe, Love

## Notebook Set Up

In [5]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

NameError: name 'MongoClient' is not defined

In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

NameError: name 'mongo' is not defined

In [8]:
# review the collections in our database
print(db.list_collection_names())

NameError: name 'db' is not defined

In [9]:
# assign the collection to a variable
establishments = db['establishments']

NameError: name 'db' is not defined

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents with a hygiene score of 20.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])

NameError: name 'establishments' is not defined

In [11]:
# Convert the result to a Pandas DataFrame
hygiene_score = pd.json_normalize(results)
hygiene_score.head()

# Display the number of rows in the DataFrame
print(f"The number of rows in the hygiene_score DataFrame is: {len(hygiene_score)}")
print("")

# Display the first 10 rows of the DataFrame
hygiene_score.head(10)

NameError: name 'pd' is not defined

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [12]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName":{'$regex': "London"}, "RatingValue" : {'$gte':4}}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} documents for establishments within 'London' with a 'RatingValue' >= 4.")

# Display the first document in the results using pprint
results = establishments.find(query)
print("")
print("The first document is:")
print("-"*50)
pprint(results[0])

NameError: name 'establishments' is not defined

In [13]:
# Convert the result to a Pandas DataFrame
london_rating_value = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(london_rating_value)}")

# Display the first 10 rows of the DataFrame
london_rating_value.to_csv("Outputs/london_rating_value.csv", index=False, header=True)

NameError: name 'pd' is not defined

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [14]:
# Search within 0.01 degree on either side of the latitude and longitude.
query = {"BusinessName": "Penang Flavours"}
fields = {"geocode.latitude": 1, "geocode.longitude": 1, "_id": 0}
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("hygiene", 1)]
limit = 5

results = list(establishments.find(query).sort(sort).limit(limit))

# Print the results
for document in results:
    pprint(document)

NameError: name 'establishments' is not defined

In [15]:
# Convert result to Pandas DataFrame
penang_geo_proximity = pd.json_normalize(results)
penang_geo_proximity

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [16]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match = {'$match': {"scores.Hygiene":0}}

# 2. Groups the matches by Local Authority
group = {'$group': {"_id" : "$LocalAuthorityName",
                   "count": {'$sum':1}}}

# 3. Sorts the matches from highest to lowest
sort = {'$sort':{"count":-1}}

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
print("")
print("The first 10 documents are:")
print("-"*50)
pprint(results[0:10])

In [17]:
# Convert the result to a Pandas DataFrame
pipeline_DF = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"The number of rows in this DataFrame is: {len(pipeline_DF)}")
print("")

# Display the first 10 rows of the DataFrame
pipeline_DF.head(10)